In [ ]:
# Important imports
import torch
import cv2
from torch import nn as nn
import numpy as np
import pandas as pd
import moviepy.editor as mp
from PIL import Image, ImageFilter
from google.colab.patches import cv2_imshow
from torchvision import transforms,models
from moviepy.editor import VideoFileClip 


#Choosing devide to be gpu if have one, else cpu
# DEVICE = torch.device('cuda' if torch.cuda.is_available else 'cpu')


In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = [0.485, 0.456, 0.406],
        std = [0.229, 0.224, 0.225]
    )
])

In [ ]:
def get_model():
    
    model = torchvision.models.resnet101(pretrained = True)
    
    for param in model.parameters():
        param.requires_grad = False
        
    nr_feat = model.fc.in_features 
    model.fc = nn.Sequential(OrderedDict([('fc',nn.Linear(nr_feat,1)),('sigmoid',nn.Sigmoid())]))

    
    return model

In [ ]:
import torchvision
from collections import OrderedDict

loaded_model = get_model()a
loaded_model.load_state_dict(torch.load(r'/content/drive/MyDrive/CVproject/resnet101_classificator.pth'), strict=False)


<All keys matched successfully>

In [ ]:

# load our serialized model and parameter from disk
modelFile = "/content/drive/MyDrive/CVproject/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "/content/drive/MyDrive/CVproject/deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

# load the input video and construct an input blob
# for the video by resizing to a fixed 300x300 pixels 
# and the normalizing it
# df = pd.DataFrame(columns=['frame', 'x_min', 'y_min', 'x_max', 'y_max'])
cap = cv2.VideoCapture('/content/drive/MyDrive/CVproject/kids.mp4')


fps = int(cap.get(cv2.CAP_PROP_FPS))

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output_kids.mp4', cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))

while(True):
    ret, img = cap.read()
    if ret == True:
        # img = cv2.resize(img, None, fx=0.5, fy=0.5)
        height, width = img.shape[:2]
        img1 = img.copy()

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # create blob from image
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)),1.0, (300, 300), (104.0, 117.0, 123.0))
        # print("First Blob: {}".format(blob.shape))
        # pass the blob through the network and obtain the detections and predictions
        net.setInput(blob)
        faces = net.forward()

        # loop over the detections(faces)
        for i in range(faces.shape[2]):
            # extract the confidence (i.e. probability) associeted with the prediction
            confidence = faces[0, 0, i, 2]

            # filter out weak detections by ensuring the `confidence` is greater than
            # the minimum confidence
            if confidence > 0.5:
                # compute the (x, y) - coordinates of the bounding box for the object
                box = faces[0, 0, i, 3:7] * np.array([width, height, width, height])

                (x, y, x1, y1) = box.astype("int")
                current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)

                # new_row = {'frame': current_frame, 'x_min': x, 'y_min': y, 'x_max': x1, 'y_max': y1}
                # df = df.append(new_row, ignore_index=True)
            
                a = (img[y:y1, x:x1]) 
            # cv2_imshow(a)
                PIL_image = Image.fromarray(a)
                tensor = transformations(PIL_image)
                prediction = loaded_model(tensor.unsqueeze(0))
                
                if float(prediction) > .5:
          # cv2.rectangle(img1, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
                  face_image = Image.fromarray(img1)
                  cropped_image = face_image.crop((x, y, x1, y1))
                  blurred_image = cropped_image.filter(ImageFilter.GaussianBlur(radius=15))
                  face_image.paste(blurred_image, (x, y, x1, y1))
                  img1 = np.array(face_image)

      # cv2_imshow(img1)
        out.write(img1)

        # cv2_imshow(img1)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
# print(df)         
cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
cap = VideoFileClip("/content/drive/MyDrive/CVproject/kids.mp4")
audio = cap.audio
out = VideoFileClip("/content/output_kids.mp4")
final_video = out.set_audio(audio)
final_video.write_videofile("final_video_kids.mp4", fps)



[MoviePy] >>>> Building video final_video_kids.mp4
[MoviePy] Writing audio in final_video_kidsTEMP_MPY_wvf_snd.mp3


100%|██████████| 191/191 [00:00<00:00, 716.49it/s]

[MoviePy] Done.
[MoviePy] Writing video final_video_kids.mp4



100%|██████████| 259/259 [00:05<00:00, 45.01it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: final_video_kids.mp4 

